In [11]:
from __future__ import print_function
import freesound
import random
import utils
import json
from IPython.core.display import display, HTML
from collections import defaultdict

API_KEY='rKDsIRp9pAJ5RzgdDZWcKfozpVisSCbs28ewvbFa'
c = freesound.FreesoundClient()
c.set_token(API_KEY,"token")
PAGE_SIZE = 50 # Page size for fs requests, no need to change that

Create Freesound Api client .

In [12]:
# Configure dataset parameters and audio categories

DATASET_NAME = 'my_amazing_dataset' # Dataset will be saved in a .json file with this name
N = 10 # Number of sounds per class (set 'None' to get all in dataset)
DATASET_CLASSES = [
    #''Percussion', 
    'Brass instrument', 
    #'Wind instrument, woodwind instrument', 
    #'Bowed string instrument', 
    'Plucked string instrument', 
    #'Singing', 
    #'Speech', 
    #'Domestic sounds, home sounds', 
    #'Vehicle', 
    #'Generic impact sounds', 
    #'Mechanisms'
]  

# Get sound examples from Freesound
data_index =  json.load(open('data_index.json'))
dataset = defaultdict(list)

for klass in DATASET_CLASSES:
    sound_ids = data_index.get(klass, None)
    if sound_ids is None:
        print('Skipping class %s as no data is available for it' % klass)
        continue
    random.shuffle(sound_ids)  # Shuffle order of sound ids
    sound_ids = sound_ids[:N]  # Limit number of selected sound ids
    
    print('Getting sounds\' data for class %s...' % klass)
    for i in range(0, len(sound_ids), PAGE_SIZE):
        current_sound_ids = sound_ids[i:i + PAGE_SIZE]
    
        fields = "id,tags,description,username,analysis"
        descriptors = "lowlevel.spectral_centroid,lowlevel.barkbands.mean"
        results_pager = c.text_search(
            filter='id:(%s)' % ' OR '.join([str(sid) for sid in current_sound_ids]),
            page_size=PAGE_SIZE,
            fields = fields, descriptors=descriptors
        )
        dataset[klass] += results_pager.results

        # TIP ON AUDIO FEATURES: you can get also audio features extracted in freesound by passing a 'descriptors' 
        # parameter in the text_search function and including 'analysis' in the fields list 
        # (see http://www.freesound.org/docs/api/resources_apiv2.html#response-sound-list):
        #
        # fields = "id,tags,description,username,analysis"
        # descriptors = "lowlevel.spectral_centroid,lowlevel.barkbands.mean"
        #
        # e.g.: results_page = c.text_search(query=target_query, ..., fields=fields, descriptors=descriptors)
        # ...

# Show information and save dataset to file so we can work with it later on
utils.save_to_json('%s.json' % DATASET_NAME, dataset)
print('\nDataset created with %i classes:' % len(dataset))
for klass, sounds in dataset.items():
    print('\t%s: %i sounds' % (klass, len(sounds)))
print('Saved to %s' % '%s.json' % DATASET_NAME)

Getting sounds' data for class Brass instrument...
Getting sounds' data for class Plucked string instrument...

Dataset created with 2 classes:
	Plucked string instrument: 10 sounds
	Brass instrument: 10 sounds
Saved to my_amazing_dataset.json


I have choosen two dataset classes : 1. Brass instrument 2.Plucked string instrument .I am downloading 10 sounds per each dataset. These two classes have sounds very different from one another so it would be nice to see the classification results.

Explore the dataset

In [13]:
# Pick some sounds from each category and show players (Freesound embeds) to listen to them
for count, (class_name, sounds) in enumerate(dataset.items()):
    html = "<h3 style='color:#bbb;'>%i) %s</h3>" % (count + 1, class_name)
    html += "<h4>Example sounds:</h4><br>"
    html += utils.generate_html_with_sound_examples([sound['id'] for sound in sounds][:6])
    html += "<h4>Most commons tags tagcloud:</h4><br>"
    class_tags = utils.get_all_tags_from_class(class_name, dataset)
    html += utils.generate_html_tagcloud(class_tags, N=100, max_px=30, min_px=10, pow_scale=1.2)
    html += "<br><br>"
    display(HTML(html)) # <- This is pure jupyter notebook AWESOMENESS which renders the HTML in the output of the cell